### LibraryをImport

In [1]:
import os
import sys

from langchain_openai import AzureChatOpenAI
sys.path.append('/home/ubuntu/oci_script/ochacafe_s9_1/langchain/libs/community')
from langchain_community.graphs.oracle_graph import OracleGraph

import oracledb
from typing import (Any, Dict, Type)

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.graph_transformers import LLMGraphTransformer
from pydantic import BaseModel, Field
from langchain_core.tools import BaseTool
from langchain.agents import AgentExecutor, create_tool_calling_agent

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

ModuleNotFoundError: No module named 'langchain_community.graphs.oracle_graph'

### 環境変数を設定

In [3]:
UN = os.environ.get("UN")
PW = os.environ.get("PW")
DSN = os.environ.get("DSN")
JDBC_URL=os.environ["JDBC"]
wallet_dir = os.environ.get('TNS_ADMIN')
wallet_pw = os.environ.get('PW')

AZURE_OPENAI_API_VERSION=os.environ["AZURE_OPENAI_API_VERSION"]
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
GRAPH_NAME=os.environ["GRAPH_NAME"]


### Oracle Connection

In [20]:
oracledb.init_oracle_client()
client = OracleGraph(url=JDBC_URL, username=UN, password=PW)

try:
    sql_connection = oracledb.connect(user=UN, password=PW, dsn=DSN, config_dir=wallet_dir, wallet_location=wallet_dir, wallet_password=wallet_pw)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
client.set_connection(client=sql_connection)


20:58:21.763 [main] DEBUG org.apache.commons.vfs2.impl.DefaultFileSystemManager -- Using "/tmp/vfs_cache-1035463264368401156" as temporary files store.
20:58:21.994 [main] DEBUG org.metaborg.spoofax.core.language.LanguageComponentFactory -- Creating language component for zip:file:///tmp/pgql.spoofax-language112681862806967389921e1d88a5-43b4-4b50-be9e-d06246e9f693!/
20:58:22.007 [main] DEBUG org.metaborg.core.language.LanguageService -- Adding language pgql-lang
20:58:22.008 [main] DEBUG org.metaborg.core.language.LanguageService -- Adding language impl. oracle.pg:pgqllang:0.0.0SNAPSHOT
20:58:22.008 [main] DEBUG org.metaborg.core.language.LanguageService -- Adding language comp. oracle.pg:pgqllang:0.0.0SNAPSHOT@zip:file:///tmp/pgql.spoofax-language112681862806967389921e1d88a5-43b4-4b50-be9e-d06246e9f693!/
20:58:22.490 [main] DEBUG org.metaborg.spoofax.core.stratego.StrategoRuntimeService -- Creating prototype runtime for language comp. oracle.pg:pgqllang:0.0.0SNAPSHOT@zip:file:///tmp/p

#### Clean up

In [5]:
drop_all_query = """
BEGIN
    FOR cur_rec IN (SELECT object_name FROM user_objects WHERE object_type = 'TABLE') LOOP
        EXECUTE IMMEDIATE 'DROP TABLE "' || cur_rec.object_name || '" CASCADE CONSTRAINTS';
    END LOOP;
END;
"""
client.sql_execute(query=drop_all_query)

2024-11-30 00:28:06,858 - ERROR - Failed to execute query: DPY-1003: the executed statement does not return rows


[]

### LLMの設定

In [6]:
llm = AzureChatOpenAI(
        api_version=AZURE_OPENAI_API_VERSION,
        azure_deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        temperature=0.0,
    )

### GraphDocumentsの作成

In [7]:
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Place", "Role"],
    allowed_relationships=["Retainer", "Resides_at", "Had_role"]
    )

text = """
織田 信長（おだ のぶなが）は、日本の戦国時代から安土桃山時代にかけての武将・大名。戦国の三英傑の一人。
尾張国（現在の愛知県）出身。織田信秀の嫡男。家督争いの混乱を収めた後に、桶狭間の戦いで敵大将の今川義元を討ち取り、勢力を拡大した。足利義昭を奉じて上洛し、後には義昭を追放することで、畿内を中心に独自の中央政権（「織田政権」[注釈 4]）を確立して天下人となった。
豊臣秀吉、徳川家康や明智光秀を家臣とし信長の政権を広げた。しかし、天正10年6月2日（1582年6月21日）、家臣・明智光秀に謀反を起こされ、本能寺で自害した。
これまで信長の政権は、豊臣秀吉による豊臣政権、徳川家康が開いた江戸幕府への流れをつくった画期的なもので、その政治手法も革新的なものであるとみなされてきた[2]。しかし、近年の歴史学界ではその政策の前時代性が指摘されるようになり、しばしば「中世社会の最終段階」とも評され[2]、その革新性を否定する研究が主流となっている
"""

documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(graph_documents)


[GraphDocument(nodes=[Node(id='織田信長', type='Person', properties={}), Node(id='尾張国', type='Place', properties={}), Node(id='愛知県', type='Place', properties={}), Node(id='織田信秀', type='Person', properties={}), Node(id='今川義元', type='Person', properties={}), Node(id='足利義昭', type='Person', properties={}), Node(id='畿内', type='Place', properties={}), Node(id='豊臣秀吉', type='Person', properties={}), Node(id='徳川家康', type='Person', properties={}), Node(id='明智光秀', type='Person', properties={}), Node(id='本能寺', type='Place', properties={})], relationships=[Relationship(source=Node(id='織田信長', type='Person', properties={}), target=Node(id='尾張国', type='Place', properties={}), type='RESIDES_AT', properties={}), Relationship(source=Node(id='尾張国', type='Place', properties={}), target=Node(id='愛知県', type='Place', properties={}), type='RESIDES_AT', properties={}), Relationship(source=Node(id='織田信長', type='Person', properties={}), target=Node(id='織田信秀', type='Person', properties={}), type='RETAINER', properties

### Oracle Graphの作成

In [8]:
client.add_graph_documents(graph_documents=graph_documents, graph_name='person_graph')

2024-11-30 00:28:19,372 - ERROR - Failed to execute query: ORA-42421: property graph does not exist
Help: https://docs.oracle.com/error-help/db/ora-42421/


### パターンマッチング

In [21]:
select_query = """
SELECT *
FROM GRAPH_TABLE( PERSON_GRAPH
MATCH (n IS Person)
COLUMNS(n.id)
)
"""
sql_results = client.sql_execute(query=select_query)
print(sql_results)

[('今川義元',), ('徳川家康',), ('明智光秀',), ('織田信秀',), ('織田信長',), ('豊臣秀吉',), ('足利義昭',)]


In [24]:
select_query = """
SELECT a, b
FROM GRAPH_TABLE( PERSON_GRAPH
MATCH (a IS Person) - [e] -> (b IS Person)
WHERE e.name = 'RETAINER' and a.id = '織田信長'
COLUMNS(a.id as a, b.id as b))
"""
sql_results = client.sql_execute(query=select_query)
print(sql_results)

[('織田信長', '織田信秀'), ('織田信長', '今川義元'), ('織田信長', '足利義昭'), ('織田信長', '豊臣秀吉'), ('織田信長', '徳川家康'), ('織田信長', '明智光秀')]


#### Tools
* RetainerTool: 部下を取得する
* ResideTool: 居住地を取得する
* RoleTool: Roleを取得する

In [38]:
class RetainerToolInput(BaseModel):
    query: str = Field(description="A query for RetainerTool")
    names: str = Field(description="Name of target person")

class RetainerTool(BaseTool):
    name: str = "retainer_tool"
    description: str ="""
    A function that identifies someone who is retainer a particular person 
    and returns the names of both individuals.
    """
    args_schema: Type[BaseModel] = RetainerToolInput

    def _run(self, names: str, *args: Any, **kwargs: Any) -> str:
        GRAPH_NAME = os.getenv("GRAPH_NAME")
        return f"""
        SELECT a, b
        FROM GRAPH_TABLE({GRAPH_NAME}
        MATCH (a IS Person) - [e] -> (b IS Person)
        WHERE e.name = 'RETAINER' and a.id = '{names}'
        COLUMNS(a.id as a, b.id as b))
        """

class RoleToolInput(BaseModel):
    query: str = Field(description="A query for RoleTool")
    names: str = Field(description="Name of target person")

class RoleTool(BaseTool):
    name: str = "role_tool"
    description: str = """
    A function that identifies a role a particular person holds 
    and returns the names of the person and the role.
    """
    args_schema: Type[BaseModel] = RoleToolInput

    def _run(self, names: str, *args: Any, **kwargs: Any) -> str:
        GRAPH_NAME = os.getenv("GRAPH_NAME")
        return f"""
        SELECT a, b
        FROM GRAPH_TABLE({GRAPH_NAME}
        MATCH (a IS Person) - [e] -> (b IS Role)
        WHERE e.name = 'HAD_ROLE' and a.id = '{names}'
        COLUMNS(a.id as a, b.id as b))
        """

class ResideToolInput(BaseModel):
    query: str = Field(description="A query for ResideTool")
    names: str = Field(description="Name of target person")

class ResideTool(BaseTool):
    name: str = "recommend_tool"
    description: str ="""
    A function that identifies a person who reside at particular place
    and returns the names of the person and the place.
    """
    args_schema: Type[BaseModel] = ResideToolInput

    def _run(self, names: str, *args: Any, **kwargs: Any) -> str:
        GRAPH_NAME = os.getenv("GRAPH_NAME")
        return f"""
        SELECT a, c
        FROM GRAPH_TABLE({GRAPH_NAME}
        MATCH (a IS Person) - [b is Resides_at] -> (c is Place)
        WHERE a.id = '{names}'
        COLUMNS(a.id as a, c.id as c))
        """

class SQLExecuteToolInput(BaseModel):
    query: str = Field(description="A query for SQLExecuteTool")

class SQLExecuteTool(BaseTool):
    name: str = "sql_execute_tool"
    description: str = """
    A function that executes SQL queries and returns the results.
    """
    args_schema: Type[BaseModel] = SQLExecuteToolInput
    
    def _run(self, query: str, *args: Any, **kwargs: Any) -> list:
        return client.sql_execute(query=query)


reside = ResideTool()
role = RoleTool()
retainer = RetainerTool()
sqlexecute = SQLExecuteTool()
tools=[reside, role, retainer, sqlexecute]

### Agentsを作成

In [34]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
print(prompt.messages)
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
sql_agent = AgentExecutor(agent=agent, tools=tools, verbose=True)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad', optional=True)]


#### Agentsを実行

In [35]:
sql_agent.invoke({'input': "織田信長の部下は誰ですか?"})


Invoking: `retainer_tool` with `{'query': '織田信長の部下', 'names': '織田信長'}`



        SELECT a, b
        FROM GRAPH_TABLE(PERSON_GRAPH
        MATCH (a IS Person) - [e] -> (b IS Person)
        WHERE e.name = 'RETAINER' and a.id = '織田信長'
        COLUMNS(a.id as a, b.id as b))
        
Invoking: `sql_execute_tool` with `{'query': "SELECT a, b FROM GRAPH_TABLE(PERSON_GRAPH MATCH (a IS Person) - [e] -> (b IS Person) WHERE e.name = 'RETAINER' and a.id = '織田信長' COLUMNS(a.id as a, b.id as b))"}`


[('織田信長', '織田信秀'), ('織田信長', '今川義元'), ('織田信長', '足利義昭'), ('織田信長', '豊臣秀吉'), ('織田信長', '徳川家康'), ('織田信長', '明智光秀')]織田信長の部下は以下の人物です：

- 織田信秀
- 今川義元
- 足利義昭
- 豊臣秀吉
- 徳川家康
- 明智光秀

> Finished chain.


{'input': '織田信長の部下は誰ですか?',
 'output': '織田信長の部下は以下の人物です：\n\n- 織田信秀\n- 今川義元\n- 足利義昭\n- 豊臣秀吉\n- 徳川家康\n- 明智光秀'}

In [37]:
sql_agent.invoke({'input': "織田信長の居住地はどこですか?"})


Invoking: `recommend_tool` with `{'query': '織田信長の居住地', 'names': '織田信長'}`



        SELECT a, c
        FROM GRAPH_TABLE(PERSON_GRAPH
        MATCH (a IS Person) - [b is Resides_at] -> (c is Place)
        WHERE a.id = 織田信長
        COLUMNS(a.id as a, c.id as c))
        
Invoking: `sql_execute_tool` with `{'query': 'SELECT a, c FROM GRAPH_TABLE(PERSON_GRAPH MATCH (a IS Person) - [b is Resides_at] -> (c is Place) WHERE a.id = 織田信長 COLUMNS(a.id as a, c.id as c))'}`




2024-12-14 21:14:12,998 - ERROR - Failed to execute query: ORA-40956: invalid property name format
Help: https://docs.oracle.com/error-help/db/ora-40956/


[]織田信長の居住地に関する情報は見つかりませんでした。他に知りたいことがあれば教えてください。

> Finished chain.


{'input': '織田信長の居住地はどこですか?',
 'output': '織田信長の居住地に関する情報は見つかりませんでした。他に知りたいことがあれば教えてください。'}